In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [85]:
noise_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/val_Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)


data_BBH_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/val_Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)


data_BNS_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/diff_val_Final_BNS_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BNS = data_BNS_df.values.astype(float)

In [86]:
noise *= 1e19
data_BBH *= 1e19
data_BNS *= 1e20

In [87]:
X = np.concatenate((noise, data_BBH, data_BNS), axis=0)

print(len(noise_df.index))
print(X)
print(X.shape)

1000
[[ 0.94702137 -0.26324898  1.33689108 ... -0.7913548   0.34316772
  -0.91804836]
 [-0.96397029  0.23194914  0.01669288 ... -1.27912926  0.13591558
  -0.36164634]
 [-0.09986968  1.85085328  0.35783558 ...  0.26078978 -0.85469271
   0.40428457]
 ...
 [-0.08780296 -0.01443747  0.00614803 ...  0.12960821  0.01767292
   0.39055772]
 [ 0.16229123 -0.24215442 -0.5537051  ... -0.13285482  0.01147719
   0.03154607]
 [-0.1017297  -0.33965818 -0.1618192  ... -0.1976131  -0.2488811
  -0.32401994]]
(3000, 16384)


In [88]:
y = [[0, -1] for i in range(1000)] + [[1, 0] for i in range(1000)] + [[1, 1] for i in range(1000)]
y = np.array(y)

In [89]:
X = np.expand_dims(X, axis=-1)

print(X.shape)
# print(X)

(3000, 16384, 1)


In [90]:
X_val, _, y_val, _ = train_test_split(X, y, test_size=1) #, random_state=1

y1 = [y_val[i,1] for i in range(len(y_val))]
y_val = [y_val[i,0] for i in range(len(y_val))]

In [91]:
model1 = load_model("/content/gdrive/MyDrive/GW_SOP/Data/multi_level_model_level_1.h5")

In [92]:
pred = model1.predict(X_val)

In [93]:
# y_pred = np.argmax(pred, axis = 1)
# for i in range(2999):
#   print(y_val[i], y_pred[i])

In [94]:
y_pred = np.argmax(pred, axis = 1)
correct = 0
correct2 = 0
X1 = []
y2 = []

for i in range(len(y_val)):
  if y_pred[i]==y_val[i]:
    correct += 1
    if y_pred[i]==0:
      correct2 += 1

  if y_pred[i]==1:
    X1.append(X_val[i])
    y2.append(y1[i])

acc = (correct/len(y_val))*100
print(f"Total no. of data samples in level 1: {len(y)}")
print(f"No. of correct predictions: {correct}")
print(f"Level 1 accuracy: {acc}")

Total no. of data samples in level 1: 3000
No. of correct predictions: 2995
Level 1 accuracy: 99.86662220740247


In [95]:
X1 = np.array(X1)
y2 = np.array(y2)
print(X1.shape)
# print(X1)

(1995, 16384, 1)


In [96]:
X1_val, _, y1_val, _ = train_test_split(X1, y2, test_size=1) #, random_state=1
print(X1_val.shape, y1_val.shape)

(1994, 16384, 1) (1994,)


In [97]:
model2 = load_model("/content/gdrive/MyDrive/GW_SOP/Data/multi_level_model_level_2.h5")

In [98]:
pred = model2.predict(X1_val)

In [99]:
y_pred = np.argmax(pred, axis = 1)
correct1 = 0

for i in range(len(y1_val)):
  if y_pred[i]==y1_val[i]:
    correct1 += 1

acc = (correct1/len(y1_val))*100
print(f"Total no. of data samples in level 2: {len(y1_val)}")
print(f"No. of correct predictions: {correct1}")
print(f"Level 2 accuracy: {acc}")

Total no. of data samples in level 2: 1994
No. of correct predictions: 1994
Level 2 accuracy: 100.0


In [100]:
final_acc = ((correct1 + correct2)/3000)*100
print(f"The final accuracy of the model is {final_acc}")

The final accuracy of the model is 99.8
